In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
import ee
import geemap.foliumap as geemap
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from keras.applications.vgg16 import preprocess_input
from keras.models import load_model

In [ ]:
ee.Authenticate()
ee.Initialize(project="215656163750")
drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/Colab Notebooks/')
import utils
import explore

Mounted at /content/drive


In [ ]:
# The fine-tuned model from initialModel.ipynb

model_type = "VGG16"
model = load_model('/content/drive/MyDrive/CAFO_models/VGG16_data_combo_1/model_ft.keras')

In [ ]:
# Functions for data preparation

data_path = "/content/drive/MyDrive/CAFO_data/forTraining/"


def prepare(dataset):

  df = pd.read_pickle(f'{data_path}{dataset}.pkl')

  images = []
  labels = []

  for idx, row in df.iterrows():
    if row['Farm type'] in ["Poultry", "Broiler", "Layer", "Pig", "Unknown CAFO"]:
      labels.append("CAFO")
      images.append(row["Sentinel"])
    elif row['Farm type'] in ["Non-farm", "Feedlot", "Dairy"]:
      labels.append("Non-farm")
      images.append(row["Sentinel"])
    else:
      print(f"Unexpected farm type, {row['Farm type']}")

  label_encoder = LabelEncoder()
  encoded_labels = label_encoder.fit_transform(labels)
  one_hot_labels = to_categorical(encoded_labels, num_classes=len(set(labels)))

  class_mapping = dict(zip(encoded_labels, labels))
  print("Class mapping:", class_mapping)

  return images, one_hot_labels, df

In [ ]:
# Main function for finding images that need checking

def find_dodgy_images(dataset, model_type, label, probability_range, ascending,\
                      results=None):

  # Get the Sentinel images etc. for this country/location
  X, y, df = prepare(dataset)
  meta = df.drop(columns=["Sentinel"])

  # Apply the model to all images, if not already done
  if results is None:
    results = utils.get_predictions(model, X, y, model_type, meta)

  # Create a new column conveniently displaying class probability
  results.loc[:, "prob"] = results.loc[:, "Model Probabilities"].str[label]

  # Identify images labeled as <label> with probability in <probability_range>
  # E.g., label=0 and probability_range=[0, 0.5] would show images labeled as
  # CAFOs but with low model CAFO probabilities
  to_check = results[(results["Label"] == label) &\
                     (results["prob"] > probability_range[0]) &\
                     (results["prob"] <= probability_range[1])]

  # Sort by probability
  to_check.sort_values(by=["prob"], ascending=ascending, inplace=True)

  print(f"There are {len(to_check)} images to check")

  return df, to_check, results

## Mexico


In [ ]:
# File we're reading in and editing
mex_in = "1"

# File that will be written out
mex_out = "2"

Start by examining images labeled not-CAFO but with a low model probability of being a not-CAFO. That is, images that might actually just be mislabelled CAFOs.

In [ ]:
mex, check_mex, predictions = find_dodgy_images(f"mex_iter{mex_in}", model_type,\
                                                label=1,\
                                                probability_range=[0.4, 0.6],\
                                                ascending=True,\
                                                results=predictions)

Class mapping: {0: 'CAFO', 1: 'Non-farm'}
There are 207 images to check


<ipython-input-5-39811bba3fc1>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_check.sort_values(by=["prob"], ascending=ascending, inplace=True)


In [ ]:
# If desired, ID images that are already known to be mislabelled, from previous
# inspections, and exclude from re-examination. This is useful if, for some
# reason, the training dataset has been recreated and includes new images.

# NOT READY YET

#gdf_list = []
#for iteration in [0]:
#  gdf_list.append(pd.read_pickle(f'{data_path}mex_iter{iteration}.pkl'))
#gdf = pd.concat(gdf_list)

#known_false_negs = gdf[gdf["Farm type"] == "Non-farm"]
#to_exclude = gpd.sjoin_nearest(check_mex, known_false_negs, how="inner", max_distance=100)
#display(to_exclude)
#print(len(to_exclude))

In [ ]:
# Examine those images, flag ones that really are CAFOs

false_negs = explore.loop_over_buildings(check_mex[:100])


Working on feature 1 of 100


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,Area (sq m),Aspect ratio,geometry,Dataset name,Farm type,Length (m),Parent coords,prob
730,1,"[0.5975785, 0.40242144]",0,México,Jalisco,Arandas,3208.7271,1.465332,"POLYGON ((-102.28454 20.6495, -102.28455 20.64...",Mexico,Non-farm,82.631467,None,0.402421


Enter reject to reject, exit to exit, or any key to continue  
Working on feature 2 of 100


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,Area (sq m),Aspect ratio,geometry,Dataset name,Farm type,Length (m),Parent coords,prob
5582,1,"[0.59732425, 0.40267572]",0,México,Jalisco,San Juan de los Lagos,5172.290039,4.582393,"POLYGON ((-102.31718 21.31759, -102.31715 21.3...",Mexico,Dairy,175.1748,"POLYGON ((-102.315893 21.317922, -102.3159 21....",0.402676


Enter reject to reject, exit to exit, or any key to continue  
Working on feature 3 of 100


...

Enter reject to reject, exit to exit, or any key to continue  
Working on feature 99 of 100


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,Area (sq m),Aspect ratio,geometry,Dataset name,Farm type,Length (m),Parent coords,prob
3350,1,"[0.491658, 0.508342]",1,México,Guanajuato,Jaral del Progreso,1214.676392,11.894232,"POLYGON ((-101.04526 20.41448, -101.04628 20.4...",Mexico,Non-farm,119.854784,None,0.508342


Enter reject to reject, exit to exit, or any key to continue  
Working on feature 100 of 100


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,Area (sq m),Aspect ratio,geometry,Dataset name,Farm type,Length (m),Parent coords,prob
947,1,"[0.49158576, 0.50841427]",1,México,Tabasco,Balancán,980.684509,2.182159,"POLYGON ((-91.24873 17.85965, -91.24873 17.859...",Mexico,Dairy,51.108217,"POLYGON ((-91.246796 17.860643, -91.246813 17....",0.508414


Enter reject to reject, exit to exit, or any key to continue  


In [ ]:
print(false_negs)

#false_negs_1 =
#false_negs = false_negs_1 +

print(f"Rejected {len(false_negs)} / {len(check_mex)} images")

[5102, 943, 6929]
Rejected 3 / 207 images


Now examine Mexico images labeled CAFO but with a low probability of being so.



In [ ]:
_, check_mex2, _ = find_dodgy_images(f"mex_iter{mex_in}", model_type, label=0,\
                                  probability_range=[0, 0.4], ascending=True,\
                                  results=predictions) #use existing predictions

Class mapping: {0: 'CAFO', 1: 'Non-farm'}
There are 262 images to check


<ipython-input-5-39811bba3fc1>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_check.sort_values(by=["prob"], ascending=ascending, inplace=True)


In [ ]:
false_pos = explore.loop_over_buildings(check_mex2[100:200])

Working on feature 1 of 100


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,Area (sq m),Aspect ratio,geometry,Dataset name,Farm type,Length (m),Parent coords,prob
7223,0,"[0.14360169, 0.8563983]",1,México,Yucatán,Umán,951.958313,7.274005,"POLYGON ((-89.73379 20.881, -89.73379 20.88175...",Mexico,Layer,82.762628,"POLYGON ((-89.73164 20.879925, -89.731659 20.8...",0.143602


Enter reject to reject, exit to exit, or any key to continue  
Working on feature 2 of 100


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,Area (sq m),Aspect ratio,geometry,Dataset name,Farm type,Length (m),Parent coords,prob
7405,0,"[0.1437465, 0.85625356]",1,México,San Luis Potosí,Villa de Reyes,2593.032471,1.509453,"POLYGON ((-100.88714 21.90365, -100.88761 21.9...",Mexico,Pig,63.609824,"POLYGON ((-100.884842 21.903896, -100.88485 21...",0.143746


Enter reject to reject, exit to exit, or any key to continue  reject
Working on feature 3 of 100


...

Enter reject to reject, exit to exit, or any key to continue  
Working on feature 67 of 100


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,Area (sq m),Aspect ratio,geometry,Dataset name,Farm type,Length (m),Parent coords,prob
1095,0,"[0.24506019, 0.7549398]",1,México,Sonora,Cajeme,886.35553,2.150988,"POLYGON ((-109.86171 27.29872, -109.86171 27.2...",Mexico,Pig,43.7484,"POLYGON ((-109.860216 27.299027, -109.860215 2...",0.24506


Enter reject to reject, exit to exit, or any key to continue  reject
Working on feature 68 of 100


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,Area (sq m),Aspect ratio,geometry,Dataset name,Farm type,Length (m),Parent coords,prob
2762,0,"[0.24927503, 0.7507249]",1,México,Durango,Gómez Palacio,848.239502,4.090529,"POLYGON ((-103.58069 25.63912, -103.5807 25.63...",Mexico,Pig,58.722096,"POLYGON ((-103.57907 25.639237, -103.579076 25...",0.249275


Enter reject to reject, exit to exit, or any key to continue  exit


In [ ]:
#print(false_pos)

false_pos_1 = [1423, 1422, 4810, 6050, 7261, 1984, 1001, 293, 5550, 7950, 2699, 6604, 3034, 2542, 2497, 3364, 6094, 4642, 2813, 7640, 4380, 1979, 2802, 1980, 2808, 5471, 6642, 6643, 3498, 6582, 3573, 3154, 3155, 2479, 3701, 6682, 259, 260, 1996, 426, 764]
false_pos_2 = [2001, 4935, 3243, 4961, 1990, 3019, 5718, 3087, 6668, 2902, 6641, 585, 4571, 6609, 3487, 6044, 6045, 4641, 2000, 3538, 6625, 6655, 373, 371, 372, 5091, 6628, 3449]
false_pos_3 = [7256, 3271, 5611, 3673, 6072, 3458, 5251, 3262, 5174, 7807, 594, 5151, 7378, 4572, 5613, 1172, 1171, 578]
false_pos_4 = [1822, 6233, 7412, 6018, 6016, 3181, 6238, 5202, 3224, 902, 6417, 2650, 2306, 3475, 4768, 5169, 3490, 1823, 842, 7423, 3761, 4682, 6240, 14, 181]
#false_pos_5 =
false_pos = false_pos_1 + false_pos_2 + false_pos_3 + false_pos_4
print(f"Rejected {len(false_pos)} / {len(check_mex2)} images")

Rejected 112 / 262 images


Change the labels of the mislabeled images in the original df, and pickle the cleaned df with a new name.

In [ ]:
df = mex.copy()
df.loc[df.index.isin(false_negs), ['Farm type']] = 'Unknown CAFO'
df.loc[df.index.isin(false_pos), ['Farm type']] = 'Non-farm'
df.to_pickle(f'{data_path}mex_iter{mex_out}.pkl')

Keep track of the locations that have been identified as mislabeled or otherwise undesirable

In [ ]:
mex_false_negs = df[df.index.isin(false_negs)]
mex_false_pos = df[df.index.isin(false_pos)]
mex_mislabeled = pd.concat([mex_false_negs, mex_false_pos])
mex_mislabeled.to_pickle(f'{data_path}mex_iter{mex_out}_mislabeled.pkl')